In [18]:
import requests, sys, os
from IPython.display import Image, display
from YandexMap import *


# Создание карты с соответствующими параметрами.
def load_map(mp):
    #ll - центр (автомат)
    #z(zoom) - масштабирование (автомат)
    #size - разрешение изображения
    #bbox - границы отображения
    print(mp.bbox_key())
    map_request = "http://static-maps.yandex.ru/1.x/?&size={sz}&l={type}&bbox={bbox}&pl=w:6,bw:3,{coords}". \
        format(type=mp.type, sz=mp.size_key(), coords=mp.line_key(), bbox=mp.bbox_key())

    response = requests.get(map_request)
    if not response:
        print("Ошибка выполнения запроса:")
        print(map_request)
        print("Http статус:", response.status_code, "(", response.reason, ")")
        sys.exit(1)

    # Запись полученного изображения в файл.
    map_file = "map.png"
    try:
        with open(map_file, "wb") as file:
            file.write(response.content)
    except IOError as ex:
        print("Ошибка записи временного файла:", ex)
        sys.exit(2)
    return map_file


def main():
    data = numpy.array([22.5, -93.5, 22.7, -93.8, 23.1, -94.6, 23.5, -95.4, 23.9, -96.3, 24.1,
                        -97.0, 24.5, -97.7, 24.9, -98.6, 25.2, -99.8]).reshape(-1, 2)
    mp = MapParams(data)
    load_map(mp)
    display(Image(filename='map.png'))


if __name__ == "__main__":
    main()

22.50,-99.80~25.20,25.20
Ошибка выполнения запроса:
http://static-maps.yandex.ru/1.x/?&size=650,450&l=map&bbox=22.50,-99.80~25.20,25.20&pl=w:6,bw:3,-93.50,22.50,-93.80,22.70,-94.60,23.10,-95.40,23.50,-96.30,23.90,-97.00,24.10,-97.70,24.50,-98.60,24.90,-99.80,25.20
Http статус: 400 ( Bad Request )


SystemExit: 1